In [1]:
import pandas as pd
import numpy as np

# Loading main.csv

First we are loading the "main.csv" file that we created in the notebook 1 (1_Raw_data_initial_reduction.ipynb). It was formed by concatenating multiple csv files into one and eliminating variables not used further in the process.

In [2]:
df = pd.read_csv('main.csv', sep=',', low_memory = False)
df.drop(labels=['Code commune'], axis=1, inplace=True)
df.head(5)

,Date mutation,Nature mutation,Valeur fonciere,Code postal,Code type local,Surface reelle bati,Nombre pieces principales
0,09/01/2014,Vente,"251500,00",1310.0,1.0,147.0,5.0
1,09/01/2014,Vente,"174500,00",1000.0,2.0,80.0,2.0
2,09/01/2014,Vente,"174500,00",1000.0,3.0,0.0,0.0
3,02/01/2014,Vente,"157500,00",1440.0,NaN,NaN,NaN
4,02/01/2014,Vente,"157500,00",1440.0,1.0,103.0,4.0


Here we convert the variable "Date mutation" into datetime type. This is to allow for easier sorting using this variable. It also allows us to simply decompose the variable into day, month, and year. We are appending these values further in this code cell.

In [3]:
def enrich_datetime(df, column='Date mutation', date_format="%d/%m/%Y"): # make more columns from datetime, so 14/6/2019 becomes d = 14, m = 6, y = 2019
    df[column] = pd.to_datetime(df[column], format="%d/%m/%Y")

    rng = range(df.shape[0])
    dmy = []
    for dt in list(df[column]):
        dmy.append([dt.day, dt.month, dt.year])
    df['year'] = [x[2] for x in dmy]
    df['month'] = [x[1] for x in dmy]
    df['day'] = [x[0] for x in dmy]
    return df

df = enrich_datetime(df, 'Date mutation')
df.head(5)

,Date mutation,Nature mutation,Valeur fonciere,Code postal,Code type local,Surface reelle bati,Nombre pieces principales,year,month,day
0,2014-01-09,Vente,"251500,00",1310.0,1.0,147.0,5.0,2014,1,9
1,2014-01-09,Vente,"174500,00",1000.0,2.0,80.0,2.0,2014,1,9
2,2014-01-09,Vente,"174500,00",1000.0,3.0,0.0,0.0,2014,1,9
3,2014-01-02,Vente,"157500,00",1440.0,NaN,NaN,NaN,2014,1,2
4,2014-01-02,Vente,"157500,00",1440.0,1.0,103.0,4.0,2014,1,2


In this cell we are eliminating any other for of transaction that is in the notary database. We are only interested in values with "vente" as they denote sales of properties.

In [4]:
def natur_mutation_vente_only(df): # Filters out anything that is not a sale (exchanges, auctions and such)
    df.drop(labels=df.loc[df['Nature mutation'] != 'Vente'].index, axis=0, inplace=True)
    df.drop(labels='Nature mutation', axis=1, inplace=True)
    return df

df = natur_mutation_vente_only(df)
df.head(5)

,Date mutation,Valeur fonciere,Code postal,Code type local,Surface reelle bati,Nombre pieces principales,year,month,day
0,2014-01-09,"251500,00",1310.0,1.0,147.0,5.0,2014,1,9
1,2014-01-09,"174500,00",1000.0,2.0,80.0,2.0,2014,1,9
2,2014-01-09,"174500,00",1000.0,3.0,0.0,0.0,2014,1,9
3,2014-01-02,"157500,00",1440.0,NaN,NaN,NaN,2014,1,2
4,2014-01-02,"157500,00",1440.0,1.0,103.0,4.0,2014,1,2


Here we are using variable "Code type local" that distinguishes between types of property. We are looking only for 1 and 2, which denotes houses and apartments.

### apartment = 2
### house = 1

In [5]:
def only_houses_and_apartment_data(df):
    df.dropna(axis=0, how='any', subset=['Code type local'], inplace=True)
    df['Code type local'] = df['Code type local'].astype('int64')
    df.drop(labels=df.loc[(df['Code type local'] != 2) & (df['Code type local'] != 1)].index, axis=0, inplace=True)
    return df

df = only_houses_and_apartment_data(df)
df.head(5)

,Date mutation,Valeur fonciere,Code postal,Code type local,Surface reelle bati,Nombre pieces principales,year,month,day
0,2014-01-09,"251500,00",1310.0,1,147.0,5.0,2014,1,9
1,2014-01-09,"174500,00",1000.0,2,80.0,2.0,2014,1,9
4,2014-01-02,"157500,00",1440.0,1,103.0,4.0,2014,1,2
6,2014-01-07,"90000,00",1000.0,2,61.0,2.0,2014,1,7
8,2014-01-11,"370000,00",1250.0,1,220.0,8.0,2014,1,11


In the cell below, we are cleaning the dataset of NaN values. We look at columns that correspond to data from Meilleurtaux and eliminate any row that is missing a value. NaN values in other, non-essential variables are permitted for now.

In [6]:
def clean_nan_from_essential_columns(df):
    df.dropna(axis=0, how='any', subset=['Nombre pieces principales', 'Date mutation', 'Valeur fonciere', 'Code postal'], inplace=True)
    df.sort_values(df.columns[0], axis=0, ascending=True, inplace=True, na_position='last')
    df.reset_index(drop=True,inplace=True)
    return df

df = clean_nan_from_essential_columns(df)
df.head()

,Date mutation,Valeur fonciere,Code postal,Code type local,Surface reelle bati,Nombre pieces principales,year,month,day
0,2014-01-01,"240000,00",83640.0,1,50.0,2.0,2014,1,1
1,2014-01-01,"43000,00",16700.0,1,87.0,3.0,2014,1,1
2,2014-01-01,"240000,00",83640.0,1,50.0,2.0,2014,1,1
3,2014-01-02,"73000,00",9220.0,1,74.0,4.0,2014,1,2
4,2014-01-02,"150000,00",80500.0,1,92.0,5.0,2014,1,2


1) When the data set loads, "Valeur fonciere" is not formatted properly. It cannot be converted to integer as is, we need to eliminate the coma and double-zero that each row contains.

2) Further, we proceed to convert for mentioned variables into integers. This should speed up the model and simplify dealing (e.g. comparisons) with these variables if needed.

3) Finally, during conversion into integers and we eliminate initial 0 from postcodes like 01001, therefore we add the zero back. Though conversion to integer then might seems unnecessary, it serves as a quick check that all values are indeed numeric and no corrupted or abnormal (CEDEX) postal codes are present.

In [7]:
def cut_of_coma(df, clmn='Valeur fonciere'): # Literally cuts out a coma from numbers (together with decimal zeroes)
    temp = []
    for each in list(df[clmn]):
        each = str(each)
        if "," in str(each):
            temp.append(each[:-3])
        else:
            temp.append(each)
    df[clmn] = temp
    return df

def into_int(df, clmns=['Valeur fonciere', 'Code postal', 'Surface reelle bati', 'Nombre pieces principales']):
    for each in clmns:
        df[each] = df[each].astype('int64') # Simply takes all number colums and converts them into integers (from strings and floats)
    return df

def add_zero_before_short_postal_codes(df):
    temp = []
    for each in list(df['Code postal']):
        if each < 10000:
            temp.append('0' + str(each))
        else:
            temp.append(str(each))
    df['Code postal'] = temp
    return df

df = cut_of_coma(df)
df = into_int(df)
df = add_zero_before_short_postal_codes(df)
df.head()

,Date mutation,Valeur fonciere,Code postal,Code type local,Surface reelle bati,Nombre pieces principales,year,month,day
0,2014-01-01,240000,83640,1,50,2,2014,1,1
1,2014-01-01,43000,16700,1,87,3,2014,1,1
2,2014-01-01,240000,83640,1,50,2,2014,1,1
3,2014-01-02,73000,09220,1,74,4,2014,1,2
4,2014-01-02,150000,80500,1,92,5,2014,1,2


### Here, we split data into meilleurtaux data - data we "pretend" is the data from meilleurtaux.com, and usefull data from the notary database

Notary data is utilized in this form in another notebook, where we preprocess other dataset and variables. Here we do not use it further, it is just to illustrate how we arrived at this data.

In [29]:
# Simulating data received from Meilleurtaux
meilleurtaux_data = df[['Nombre pieces principales', 'Date mutation', 'Valeur fonciere', 'Code postal', 'Code type local', 'year', 'month', 'day']].copy()

# Data from notary database used to calculate additional variables used in the model
notary_data = df[['Code postal', 'year', 'Code type local', 'Nombre pieces principales', 'Valeur fonciere', 'Surface reelle bati']].copy()

In [9]:
meilleurtaux_data.head()

,Nombre pieces principales,Date mutation,Valeur fonciere,Code postal,Code type local,year,month,day
0,2,2014-01-01,240000,83640,1,2014,1,1
1,3,2014-01-01,43000,16700,1,2014,1,1
2,2,2014-01-01,240000,83640,1,2014,1,1
3,4,2014-01-02,73000,09220,1,2014,1,2
4,5,2014-01-02,150000,80500,1,2014,1,2


In [10]:
print('Number of unique postal codes in the notary database after cleaning: ', len(list(notary_data['Code postal'].unique())))

Number of unique postal codes in the notary database after cleaning:  5868


# Geo-locatioon data set

To avoid using postal codes (unordered categorical data), we use longitude and latitude assigned to these postal codes. We use data set available from https://www.aggdata.com/free/france-postal-codes
We are only interested in using the postal codes (for matching) and longitude, latitude. Any other column is eliminated immediately after loading.

In [11]:
def add_zero_before_short_postal_codes_2(df):
    temp = []
    for each in list(df['Postal Code']):
        if each < 10000:
            temp.append('0' + str(each))
        else:
            temp.append(str(each))
    df['Postal Code'] = temp
    return df

geo_data = pd.read_csv('fr_postal_codes.csv', encoding='latin-1')
geo_data.drop(labels=['Place Name', 'State', 'County', 'City'], axis=1, inplace=True)

geo_data.drop(labels=geo_data[~geo_data['Postal Code'].apply(lambda x: str(x).isdigit())].index, axis=0, inplace=True)
geo_data['Postal Code'] = geo_data['Postal Code'].astype('int64')
geo_data = add_zero_before_short_postal_codes_2(geo_data)

'''print('Number of rows before deleting postal codes not present in notary databse: ', len(geo_data['Postal Code'].unique()))
geo_data.drop(labels=geo_data.loc[~geo_data['Postal Code'].isin(postal_codes)].index, axis=0, inplace=True)
geo_data.reset_index(drop=True,inplace=True)
print('Number of rows after: ', len(geo_data['Postal Code'].unique()))'''

geo_data.head(5)

,Postal Code,Latitude,Longitude
0,24000,45.1833,0.7167
28,24100,44.8547,0.5658
37,24110,45.0828,0.5396
42,24120,45.1896,1.2048
45,24130,44.8667,0.2500


This commented section holds a code that can load different geo-location data set for postal codes. We decided to use the code above as it is more effective (no need to split columns) and number of postal codes present in the data set is comparable (~6000).

In [12]:
# Alternative data source for Postal Codes
# https://public.opendatasoft.com/explore/dataset/correspondance-code-insee-code-postal/information/?flg=fr

'''
def add_zero_before_short_postal_codes_2(df):
    temp = []
    for each in list(df['Postal Code']):
        if each < 10000:
            temp.append('0' + str(each))
        else:
            temp.append(str(each))
    df['Postal Code'] = temp
    return df

geo_data = pd.read_csv('correspondance-code-insee-code-postal.csv', sep=';', usecols=['Code Postal', 'geo_point_2d'])

new = geo_data['geo_point_2d'].str.split(",", n = 1, expand = True)
geo_data['Postal Code'] = geo_data['Code Postal'].copy()
geo_data['Longitude'] = new[0]
geo_data['Latitude'] = new[1]
geo_data.drop(labels=['Code Postal', 'geo_point_2d'], axis=1, inplace=True)


geo_data.drop(labels=geo_data[~geo_data['Postal Code'].apply(lambda x: str(x).isdigit())].index, axis=0, inplace=True)
geo_data['Postal Code'] = geo_data['Postal Code'].astype('int64')
geo_data = add_zero_before_short_postal_codes_2(geo_data)

print('Number of rows before deleting postal codes not present in notary databse: ', len(geo_data['Postal Code'].unique()))
geo_data.drop(labels=geo_data.loc[~geo_data['Postal Code'].isin(postal_codes)].index, axis=0, inplace=True)
geo_data.reset_index(drop=True,inplace=True)
print('Number of rows after: ', len(geo_data['Postal Code'].unique()))

geo_data.head(5)
'''

'\ndef add_zero_before_short_postal_codes_2(df):\n    temp = []\n    for each in list(df[\'Postal Code\']):\n        if each < 10000:\n            temp.append(\'0\' + str(each))\n        else:\n            temp.append(str(each))\n    df[\'Postal Code\'] = temp\n    return df\n\ngeo_data = pd.read_csv(\'correspondance-code-insee-code-postal.csv\', sep=\';\', usecols=[\'Code Postal\', \'geo_point_2d\'])\n\nnew = geo_data[\'geo_point_2d\'].str.split(",", n = 1, expand = True)\ngeo_data[\'Postal Code\'] = geo_data[\'Code Postal\'].copy()\ngeo_data[\'Longitude\'] = new[0]\ngeo_data[\'Latitude\'] = new[1]\ngeo_data.drop(labels=[\'Code Postal\', \'geo_point_2d\'], axis=1, inplace=True)\n\n\ngeo_data.drop(labels=geo_data[~geo_data[\'Postal Code\'].apply(lambda x: str(x).isdigit())].index, axis=0, inplace=True)\ngeo_data[\'Postal Code\'] = geo_data[\'Postal Code\'].astype(\'int64\')\ngeo_data = add_zero_before_short_postal_codes_2(geo_data)\n\nprint(\'Number of rows before deleting postal codes

In [13]:
postal_codes = list(geo_data['Postal Code'].unique())
print('Number of unique postal codes in the longtitude/latitude dataset after cleaning: ', len(postal_codes))

Number of unique postal codes in the longtitude/latitude dataset after cleaning:  6061


# Population data set

Here we start using a population data we compiled in another notebook. This data set contains postal codes, INSEE geo codes, population, incomes and debtors in cities, and ratios of population living in 4 different types of density of habitation. This data is compiled and further information can be found in notebooks containing the code.

In this code, we decided not to use city names, geo codes and departement codes. However, we included them if it would be decided further on that they hold value and could be useful.

In [14]:
def add_zero_before_short_postal_codes_3(df):
    temp = []
    for each in list(df['Code_postal']):
        if each < 10000:
            temp.append('0' + str(each))
        else:
            temp.append(str(each))
    df['Code_postal'] = temp
    return df

pop_data = pd.read_csv('final_dataset_pop_fr_social_indcs.csv', index_col=0)
pop_data.drop(labels=['CODGEO', 'DEP', 'Nom_commune'], axis=1, inplace=True)
pop_data.drop(labels=pop_data.loc[pop_data.duplicated(keep='first')].index, axis=0, inplace=True)

pop_data = add_zero_before_short_postal_codes_3(pop_data)

print('Number of rows before deleting postal codes not present in the longtitude/latitude dataset: ', len(pop_data['Code_postal'].unique()))
pop_data.drop(labels=pop_data.loc[~pop_data['Code_postal'].isin(postal_codes)].index, axis=0, inplace=True)
pop_data.reset_index(drop=True,inplace=True)
print('Number of rows after: ', len(pop_data['Code_postal'].unique()))

pop_data.head(5)

Number of rows before deleting postal codes not present in the longtitude/latitude dataset:  6097
Number of rows after:  6027


,Code_postal,P15_POP,MED14,nombre de redevables,Typo degr̩ de densit̩,Part population dense (1),Part population interm̩diaire (2),Part population peu dense (3),Part population tr̬s peu dense (4)
0,01400,767,21576.7,0.0,3.0,0.0,0.000000,0.887569,0.112431
1,01640,241,21672.9,0.0,4.0,0.0,0.000000,0.009166,0.990834
2,01500,14127,19756.1,0.0,2.0,0.0,0.956919,0.034704,0.008377
3,01330,1619,23204.8,0.0,3.0,0.0,0.000000,0.961197,0.038803
4,01300,109,22157.5,0.0,4.0,0.0,0.000000,0.000000,1.000000


## Aggregation of city data

Starting here, we begin with aggregation of city data into a single record for the postal code (i.e. all the data will be aggregated to a set of unique postall codes)

In [15]:
pop_data_agg = pd.DataFrame()
pop_data_agg['Code postal'] = pop_data['Code_postal'].unique()
pop_data_agg.head()

,Code postal
0,01400
1,01640
2,01500
3,01330
4,01300


Aggregation itself is done with following algorithm:

1) Unique postal codes are iterrated through

2) A cut of the data frame is made: all rows with current postal codes are picked

3) On this cut, sum of the population is made (from individual rows)

4) Every other variable is then averaged, using weighted average with population of individual cities serving as the weights.

5) This aggregated row is then appended into a temporary list which is then added into the data frame "pop_data_agg"

In [16]:
aggregated = []
for i, m in enumerate(pop_data_agg['Code postal']):
    temp = pop_data[pop_data['Code_postal'] == m]
    agg = []
    agg.append(sum(temp['P15_POP']))
    for cl in temp.columns[2:]:
        if cl in ['MED14', 'nombre de redevables', 'Typo degr̩ de densit̩']:
            agg.append(int(np.average(temp[cl], weights=temp['P15_POP'])))
        else:
            agg.append(round(float(np.average(temp[cl], weights=temp['P15_POP'])),3))
    aggregated.append(agg)
    
for i, m in enumerate(pop_data.columns[1:]):
    pop_data_agg[m] = [x[i] for x in aggregated]

del aggregated
del agg

pop_data_agg.head()

,Code postal,P15_POP,MED14,nombre de redevables,Typo degr̩ de densit̩,Part population dense (1),Part population interm̩diaire (2),Part population peu dense (3),Part population tr̬s peu dense (4)
0,01400,11151,20841,0,3,0.0,0.000,0.861,0.139
1,01640,4456,20896,0,3,0.0,0.000,0.817,0.183
2,01500,24400,20599,0,2,0.0,0.726,0.262,0.012
3,01330,8257,22493,0,3,0.0,0.000,0.800,0.200
4,01300,21063,20054,0,2,0.0,0.342,0.516,0.142


# Averages from notary database

This is a test of inflation in prices in the notary database. The result you see here are not used, as they are global (for whole France) and any more granular information is not whole (i.e. not for every postal code, departement or other regional unit). This would complicate further data cleaning and we believe including price inflation correction is not a benefitial move overall, unless better data is provided (perhaps a data set by recognized authority announcing changes of price per meter squared per year per departement).

In [30]:
notary_data.head()

,Code postal,year,Code type local,Nombre pieces principales,Valeur fonciere,Surface reelle bati
0,83640,2014,1,2,240000,50
1,16700,2014,1,3,43000,87
2,83640,2014,1,2,240000,50
3,09220,2014,1,4,73000,74
4,80500,2014,1,5,150000,92


In [31]:
notary_data.drop(labels=notary_data.loc[notary_data['Surface reelle bati'] == 0].index, axis=0, inplace=True)
notary_data.drop(labels=notary_data.loc[notary_data['Valeur fonciere'] == 0].index, axis=0, inplace=True)

notary_data['price_per_meter2'] = np.array(notary_data['Valeur fonciere'])/np.array(notary_data['Surface reelle bati'])

notary_data.head(10)

,Code postal,year,Code type local,Nombre pieces principales,Valeur fonciere,Surface reelle bati,price_per_meter2
0,83640,2014,1,2,240000,50,4800.000000
1,16700,2014,1,3,43000,87,494.252874
2,83640,2014,1,2,240000,50,4800.000000
3,09220,2014,1,4,73000,74,986.486486
4,80500,2014,1,5,150000,92,1630.434783
5,09000,2014,1,5,165000,183,901.639344
6,30360,2014,1,7,195000,123,1585.365854
7,30360,2014,1,7,195000,123,1585.365854
8,91620,2014,1,3,255000,80,3187.500000
9,56170,2014,1,0,390000,316,1234.177215


In [35]:
def reject_outliers(data, m=2):
    return data[abs(data - np.mean(data)) < m * np.std(data)]

price_2014 = reject_outliers(notary_data[notary_data['year'] == 2014]['price_per_meter2'])
price_2015 = reject_outliers(notary_data[notary_data['year'] == 2015]['price_per_meter2'])
price_2016 = reject_outliers(notary_data[notary_data['year'] == 2016]['price_per_meter2'])
price_2017 = reject_outliers(notary_data[notary_data['year'] == 2017]['price_per_meter2'])
price_2018 = reject_outliers(notary_data[notary_data['year'] == 2018]['price_per_meter2'])

inflation = [np.median(price_2015)/np.median(price_2014), np.median(price_2016)/np.median(price_2014),
      np.median(price_2017)/np.median(price_2014), np.median(price_2018)/np.median(price_2014)]
print(inflation)

[0.986156888595913, 1.0058436815193572, 1.0321190755973364, 1.0135135135135136]


### As said above, we do not use these numbers to correct for inflation as we are not confident enough it is true value of inflation. However we provide the numbers here for further use if the decision is changed.

# Data matching

In this part of the notebook we match and concatenate data together from all 3 source (notary, geo-location, and population data sets). We use postal codes as the primary key through which we connect the data together.

In the cell below we iterrate aggregated population data set and add longitude and latitude instead of postal code (which will be deleted later).

In [36]:
lon = []
lat = []
temp = list(geo_data['Postal Code'])
lo = list(geo_data['Longitude'])
la = list(geo_data['Latitude'])

for i, m in enumerate(pop_data_agg['Code postal']):
    if m in temp:
        k = temp.index(m)
        lon.append(round(lo[k],3))
        lat.append(round(la[k],3))


pop_data_agg['Longitude'] = lon
pop_data_agg['Latitude'] = lat

del lon
del lat
del lo
del la

pop_data_agg.head()

,Code postal,P15_POP,MED14,nombre de redevables,Typo degr̩ de densit̩,Part population dense (1),Part population interm̩diaire (2),Part population peu dense (3),Part population tr̬s peu dense (4),Longitude,Latitude
0,01400,11151,20841,0,3,0.0,0.000,0.861,0.139,5.080,46.158
1,01640,4456,20896,0,3,0.0,0.000,0.817,0.183,5.428,46.005
2,01500,24400,20599,0,2,0.0,0.726,0.262,0.012,5.340,45.938
3,01330,8257,22493,0,3,0.0,0.000,0.800,0.200,4.929,46.019
4,01300,21063,20054,0,2,0.0,0.342,0.516,0.142,5.712,45.717


Here we match the Meilleurtaux data (in reality from notary database) with the above used aggregated population data set. We use merge function with left-join. This should ensure that every row in Meilleurtaux data is paired with aggregated population data as long as such postal code is in both data sets. Some are not and metrics showing how many and sampling which ones are shown below.

In [37]:
meilleurtaux_data = meilleurtaux_data.merge(pop_data_agg, how='left', on='Code postal')
meilleurtaux_data.head()

,Nombre pieces principales,Date mutation,Valeur fonciere,Code postal,Code type local,year,month,day,P15_POP,MED14,nombre de redevables,Typo degr̩ de densit̩,Part population dense (1),Part population interm̩diaire (2),Part population peu dense (3),Part population tr̬s peu dense (4),Longitude,Latitude
0,2,2014-01-01,240000,83640,1,2014,1,1,7459.0,22316.0,0.0,2.0,0.0,0.624,0.364,0.012,5.717,43.333
1,3,2014-01-01,43000,16700,1,2014,1,1,9611.0,18267.0,0.0,3.0,0.0,0.000,0.707,0.293,0.162,45.959
2,2,2014-01-01,240000,83640,1,2014,1,1,7459.0,22316.0,0.0,2.0,0.0,0.624,0.364,0.012,5.717,43.333
3,4,2014-01-02,73000,09220,1,2014,1,2,1157.0,18233.0,0.0,3.0,0.0,0.000,0.865,0.135,1.564,42.764
4,5,2014-01-02,150000,80500,1,2014,1,2,12900.0,18641.0,0.0,2.0,0.0,0.450,0.354,0.197,2.567,49.650


Rows with postal codes missing in the other data set.

In [38]:
meilleurtaux_data[meilleurtaux_data.isna().any(axis=1)].head()

,Nombre pieces principales,Date mutation,Valeur fonciere,Code postal,Code type local,year,month,day,P15_POP,MED14,nombre de redevables,Typo degr̩ de densit̩,Part population dense (1),Part population interm̩diaire (2),Part population peu dense (3),Part population tr̬s peu dense (4),Longitude,Latitude
193,5,2014-01-02,242500,74960,2,2014,1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
564,2,2014-01-02,190000,97190,2,2014,1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
718,3,2014-01-02,110000,97233,2,2014,1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
995,3,2014-01-03,280000,20147,1,2014,1,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1159,3,2014-01-03,280000,20147,1,2014,1,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
print('Sample of postal codes where data enrichment failed: ', list(meilleurtaux_data[meilleurtaux_data.isna().any(axis=1)]['Code postal'])[:20])

Sample of postal codes where data enrichment failed:  ['74960', '97190', '97233', '20147', '20147', '74960', '74960', '97429', '97121', '97231', '97234', '97100', '97126', '97100', '97190', '97190', '97400', '97400', '97229', '97213']


In [40]:
print('Number of rows where data enrichment failed: ', len(meilleurtaux_data[meilleurtaux_data.isna().any(axis=1)]))
print('Number of rows where it succeeded: ', len(meilleurtaux_data[~meilleurtaux_data.isna().any(axis=1)]))
print('Percentage of failed rows: ', round(len(meilleurtaux_data[~meilleurtaux_data.isna().any(axis=1)])/len(meilleurtaux_data),4))

Number of rows where data enrichment failed:  56694
Number of rows where it succeeded:  4769574
Percentage of failed rows:  0.9883


Dropping rows with missing data.

In [41]:
meilleurtaux_data = meilleurtaux_data.dropna(axis=0)
meilleurtaux_data[meilleurtaux_data.isna().any(axis=1)].head()

,Nombre pieces principales,Date mutation,Valeur fonciere,Code postal,Code type local,year,month,day,P15_POP,MED14,nombre de redevables,Typo degr̩ de densit̩,Part population dense (1),Part population interm̩diaire (2),Part population peu dense (3),Part population tr̬s peu dense (4),Longitude,Latitude


Dropping columns used to match and sort, which are no longer useful.

In [42]:
meilleurtaux_data = meilleurtaux_data.drop(labels=['Date mutation', 'Code postal'], axis=1)
meilleurtaux_data.head()

,Nombre pieces principales,Valeur fonciere,Code type local,year,month,day,P15_POP,MED14,nombre de redevables,Typo degr̩ de densit̩,Part population dense (1),Part population interm̩diaire (2),Part population peu dense (3),Part population tr̬s peu dense (4),Longitude,Latitude
0,2,240000,1,2014,1,1,7459.0,22316.0,0.0,2.0,0.0,0.624,0.364,0.012,5.717,43.333
1,3,43000,1,2014,1,1,9611.0,18267.0,0.0,3.0,0.0,0.000,0.707,0.293,0.162,45.959
2,2,240000,1,2014,1,1,7459.0,22316.0,0.0,2.0,0.0,0.624,0.364,0.012,5.717,43.333
3,4,73000,1,2014,1,2,1157.0,18233.0,0.0,3.0,0.0,0.000,0.865,0.135,1.564,42.764
4,5,150000,1,2014,1,2,12900.0,18641.0,0.0,2.0,0.0,0.450,0.354,0.197,2.567,49.650


Saving as csv for further use in the ultimate notebook.

In [43]:
meilleurtaux_data.to_csv('final.csv')